In [11]:
import pandas as pd

Открываем датасет. Заметим, что в сsv файле

In [12]:
df_train = pd.read_csv("C://Users//yegor//Desktop//datatech_internship//model//X_data.csv",sep = ";").rename(columns = {"Unnamed: 0":"Date"})

In [13]:
df_train.head(10)

,Date,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
0,01.01.2015 0:00,212,210,211,347,353,347,474,473,481,346,348,355,241,241,243,167.85,9.22
1,01.01.2015 0:01,212,211,211,346,352,346,475,473,481,349,348,355,241,241,243,162.51,9.22
2,01.01.2015 0:02,212,211,211,345,352,346,476,473,481,352,349,355,242,241,242,164.99,9.22
3,01.01.2015 0:03,213,211,211,344,351,346,477,473,481,355,349,355,242,241,242,167.34,9.22
4,01.01.2015 0:04,213,211,211,343,350,346,478,473,482,358,349,355,243,241,242,163.04,9.22
5,01.01.2015 0:05,213,211,211,342,350,346,479,473,482,360,349,354,243,241,242,163.81,9.22
6,01.01.2015 0:06,213,212,211,341,349,346,480,473,482,363,350,354,244,241,242,166.14,9.22
7,01.01.2015 0:07,213,212,211,340,348,345,482,473,482,365,350,354,244,241,242,164.38,9.22
8,01.01.2015 0:08,213,212,211,339,347,345,483,473,482,367,350,354,244,241,242,163.89,9.22
9,01.01.2015 0:09,213,212,211,338,347,345,484,473,483,369,350,354,245,241,242,162.82,9.22


In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 18 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   Date        1048575 non-null  object 
 1   T_data_1_1  1048575 non-null  int64  
 2   T_data_1_2  1048575 non-null  int64  
 3   T_data_1_3  1048575 non-null  int64  
 4   T_data_2_1  1048575 non-null  int64  
 5   T_data_2_2  1048575 non-null  int64  
 6   T_data_2_3  1048575 non-null  int64  
 7   T_data_3_1  1048575 non-null  int64  
 8   T_data_3_2  1048575 non-null  int64  
 9   T_data_3_3  1048575 non-null  int64  
 10  T_data_4_1  1048575 non-null  int64  
 11  T_data_4_2  1048575 non-null  int64  
 12  T_data_4_3  1048575 non-null  int64  
 13  T_data_5_1  1048575 non-null  int64  
 14  T_data_5_2  1048575 non-null  int64  
 15  T_data_5_3  1048575 non-null  int64  
 16  H_data      1048575 non-null  float64
 17  AH_data     1048575 non-null  float64
dtypes: float64(2), int64(1